In [1]:
import MetricCalculatorFunctions as mcf
import numpy as np
import xarray as xr
import metpy.calc as mpcalc
import metpy.constants as mpconstants
import metpy
from metpy.units import units
from tqdm import tqdm

In [2]:
y1 = 1979
y2 = 2010

hours=3

level = 850

T = xr.open_mfdataset('/home/swenson/era5_data_direct_1.0degree_Version3/Temperature_'+str(level)+'hPa/*.nc',concat_dim='time',combine='by_coords')
T = T.resample(time=str(hours)+"H").nearest()
SPH = xr.open_mfdataset('/home/swenson/era5_data_direct_1.0degree_Version3/specific_humidity_'+str(level)+'hPa/*.nc',concat_dim='time',combine='by_coords')
SPH = SPH.resample(time=str(hours)+"H").nearest()

In [3]:
def dummy(sph,T,p):

    dewpoint = mpcalc.dewpoint_from_specific_humidity(p, T, sph)
#     The warning about RH over 120% seems to just be true?
    EQtheta = mpcalc.equivalent_potential_temperature(p, T, dewpoint)

    DX,DY = mpcalc.lat_lon_grid_deltas(T['longitude'], T['latitude'], x_dim=- 1, y_dim=- 2, geod=None)
    
    return dewpoint,EQtheta,DY,DX
    
#     return mpcalc.gradient(EQtheta,axes=[-2,-1],deltas=[DY,DX])

In [6]:
dewpoint,EQtheta,DY,DX = dummy(SPH['q'],T['t'],level*units('hPa'))

/home/swenson/anaconda3/envs/heatwave/lib/python3.6/site-packages/metpy/calc/thermo.py:1022: UserWarning: Relative humidity >120%, ensure proper units.
  warnings.warn('Relative humidity >120%, ensure proper units.')


In [15]:
dx,dy = mpcalc.lat_lon_grid_deltas(T['longitude'], T['latitude'], x_dim=- 1, y_dim=- 2)
ddx = np.broadcast_to(dx,(EQtheta.shape[0],dx.shape[0],dx.shape[1]))
ddy = np.broadcast_to(dy,(EQtheta.shape[0],dy.shape[0],dy.shape[1]))

In [16]:
a = mpcalc.gradient(EQtheta,axes=[-2,-1],deltas=[ddy,ddx]) #it works needed 3D deltas

In [ ]:
a[0]

In [12]:
dx.shape

(30, 69)

In [13]:
dy.shape

(29, 70)